# ids projekti, kivaa :---)

tästä alkaa suuri seikkailu. Olkaamme innokaita ja tehokkaita. 

In [1]:
import json
import pandas as pd
import numpy as np
import urllib.request as ur
import requests
from datetime import timedelta, date

In [2]:
# Passes correct credentials for MySportsFeeds.
# path = user credentials -file path
def authorize():
    path = 'APIauth.txt'
    temp = open(path,'r').read().splitlines()
    return (temp[0], temp[1])

In [3]:
AUTH = authorize()

In [4]:
def date_range(n):
    date_list = []

    for single_date in (date.today() - timedelta(n+1) for n in range(n)):
        date_list.append(single_date.strftime('%Y%m%d'))

    return(date_list)

In [14]:
def fetch_game_scoreboards():
    games_lists = []

    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/scoreboard.json?fordate='
    
    DATE_RANGE = 5
    for date in date_range(DATE_RANGE):
        
        json = requests.get(url + date, auth=AUTH).json()
        
        if 'gameScore' in json['scoreboard']:
            games_lists.append(json['scoreboard']['gameScore'])
            
    return(games_lists)

In [15]:
scoreboard_data = fetch_game_scoreboards()

In [16]:
def fetch_game_ids():
    game_ids = []
    
    for day in scoreboard_data:
        for data in day:
            game_ids.append(data['game']['ID'])
            
    return(game_ids)

In [17]:
lineups_url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/game_startinglineup.json?gameid='
players_json = requests.get('https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/roster_players.json?fordate=20171025', auth=AUTH).json()

# Returns player data corresponding all given game ids.
# Returns -> gameID:[home_rookies, away_rookies, home_avg_weight, away_avg_weight, home_avg_age, away_avg_age]
def fetch_player_data(game_ids):
    all_games = {}
    for id in game_ids:
        game = []
        
        # Game lineups (away and home) per game id. 
        lineups_json = requests.get(lineups_url + id, auth=AUTH).json()
        home_lineup = lineups_json['gamestartinglineup']['teamLineup'][1]['expected']['starter']
        away_lineup = lineups_json['gamestartinglineup']['teamLineup'][0]['expected']['starter']

        # Rookies per team.
        home_rookies = 0
        away_rookies = 0

        # Average team player weight.
        home_avg_weight = 0
        home_size_weight = 20
        away_avg_weight = 0
        away_size_weight = 20

        # Average team player age.
        home_avg_age = 0
        home_size_age = 20
        away_avg_age = 0
        away_size_age = 20

        # Home lineup player data per game id.
        for lineup_player in home_lineup:
            for player in players_json['rosterplayers']['playerentry']:
                if (player['player']['ID'] == lineup_player['player']['ID']):

                    if ('Age' in player['player']):
                        home_avg_age += int(player['player']['Age'])
                    else:
                        home_size_age -= 1

                        
                    if ('Weight' in player['player']):
                        home_avg_weight += int(player['player']['Weight'])
                    else:
                        home_size_weight -= 1

                    if (player['player']['IsRookie'] == 'true'):
                        home_rookies += 1
                        
        # Away lineup player data per game id.
        for lineup_player in away_lineup:
            for player in players_json['rosterplayers']['playerentry']:
                if (player['player']['ID'] == lineup_player['player']['ID']):

                    if ('Age' in player['player']):
                        away_avg_age += int(player['player']['Age'])
                    else:
                        away_size_age -= 1

                    if ('Weight' in player['player']):
                        away_avg_weight += int(player['player']['Weight'])
                    else:
                        away_size_weight -= 1

                    if (player['player']['IsRookie'] == 'true'):
                        away_rookies += 1

        # Add game data to it's array. Finally add the game to all_games by it's game id.
        game.append(home_rookies)
        game.append(away_rookies)
        
        game.append(round(home_avg_weight/float(home_size_weight)))
        game.append(round(away_avg_weight/float(away_size_weight)))
        
        game.append(round(home_avg_age/float(home_size_age)))
        game.append(round(away_avg_age/float(away_size_age)))
        
        all_games[id] = game
        
    return(all_games)    

In [26]:
# Create new file with name {path} or re-write existing one with this name.
def write_string_to_file(string, path):
    open(path,'w').write(string)

In [27]:
all_games = fetch_player_data(fetch_game_ids())
save_data_to_file(str(all_games), 'player_data.txt')